In [10]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### Prepare Dataset

In [6]:
# Define transformations for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per ResNet's pre-trained model
])

# Load datasets
train_dataset = datasets.ImageFolder(root='/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/Train', transform=transform)
val_dataset = datasets.ImageFolder(root='/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/Val', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Get class names
class_names = train_dataset.classes
print(f"Classes: {class_names}")

Classes: ['Covid', 'Normal']


### Load ResNet

In [12]:
# Load pre-trained ResNet
model = models.resnet18(pretrained=True)

# Modify the fully connected layer
num_features = model.fc.in_features  # Number of input features to the last layer
model.fc = nn.Linear(num_features, len(class_names))  # Match the number of output classes

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Loss and Optimizer

In [17]:
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimizer

### Train Model

In [25]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

Epoch 1/10, Loss: 0.1823594308426154
Epoch 2/10, Loss: 0.0797183578713508
Epoch 3/10, Loss: 0.04413909970678325
Epoch 4/10, Loss: 0.025993637282979386
Epoch 5/10, Loss: 0.025425616746110005
Epoch 6/10, Loss: 0.011492714120761556
Epoch 7/10, Loss: 0.004247002885049683
Epoch 8/10, Loss: 0.0008526597959707001
Epoch 9/10, Loss: 0.0004709275961996072
Epoch 10/10, Loss: 0.00016206290672863392


### Evaluate Model

In [27]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient computation
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

Validation Accuracy: 62.07%


### Save the Model

In [29]:
torch.save(model.state_dict(), "resnet_model.pth")
print("Model saved as resnet_model.pth")

Model saved as resnet_model.pth


### Load and Predict

In [49]:
from PIL import Image

# Load the saved model
model.load_state_dict(torch.load("resnet_model.pth"))
model.eval()

def img_passed(img_p):
    # Predict on a single image
    img_path = "/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/single_prediction/" + img_p
    # img = Image.open(img_path)
    img = Image.open(img_path).convert('RGB')
    img = transform(img).unsqueeze(0)  # Apply transformations and add batch dimension
    
    # Move to device and predict
    img = img.to(device)
    output = model(img)
    _, predicted_class = torch.max(output, 1)
    print(f"Predicted Class: {class_names[predicted_class.item()]}")

img_passed("normal.png")

Predicted Class: Normal
